# Test Misc Functions
This notebook contains snippets of code used to test various functions. 

Some functions take the source data as input to perform operations. This assumes that the data processing has been done, please check paths.

Some functions generate output files, which are stored in the folder data/samples/<some name>. If this does not exist, an error will be thrown, generate this folder first.

## Test Edge Detection on Depth

In [ ]:
%matplotlib inline 

import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from data_loader import Dataset
import OpenEXR, Imath
from scipy.misc import imsave

import sys
sys.path.append('../pytorch-normals/')
import data_processing_script

class OPT():
    def __init__(self):
        self.dataroot = './data/'
        self.file_list = './data/datalist'
        self.batchSize = 32
        self.shuffle = True
        self.phase = 'train'
        self.num_epochs = 500
        self.imsize = 224
        self.num_classes = int(3)
        self.gpu = '0'
        self.logs_path = 'logs/exp9'
        self.use_pretrained = False

opt = OPT()

###################### DataLoader #############################
dataloader = Dataset(opt)
display_output = True
save_output = False

depth_path = 'data/source-files/depth-imgs/%09d-depth.exr'
edges_output_path = 'data/samples/edges-depth-imgs/%09d-depth-edges.jpg'

if not (os.path.isdir(os.path.dirname(depth_path))):
    raise ValueError('The folder %s does not exist. Please check path.'%(os.path.dirname(depth_path)))
if(save_output):
    if not (os.path.isdir(os.path.dirname(edges_output_path))):
        raise ValueError('The folder %s does not exist. Please check path.'%(os.path.dirname(edges_output_path)))

for i in range (1,100):
    # Load Depth Img and Apply Blur
    print('Loading img %d'%(i))
    depth_img_orig = data_processing_script.exr_loader(depth_path%(i), ndim=1)
    depth_img_orig = cv2.GaussianBlur(depth_img_orig,(5,5),0)
    
    # Make all depth values greater than 2.5m as 0 (for masking edge matrix)
    depth_img_mask = depth_img_orig.copy()
    depth_img_mask[depth_img_mask > 2.5]=0
    depth_img_mask[depth_img_mask > 0 ]=1
    
    # Apply Laplacian filters for edge detection
    edges_lap = cv2.Laplacian(depth_img_orig, cv2.CV_64F, ksize=7, borderType=0 )
    edges_lap = (np.absolute(edges_lap).astype(np.uint8))
    edges_lap_binary = edges_lap.copy()
    print(edges_lap_binary.max())
    edges_lap_binary[edges_lap_binary>1] = 255
    edges_lap_binary[edges_lap_binary<=1] = 0
    edges_lap_binary = edges_lap_binary * depth_img_mask
    
    # Apply Canny filter
    depth_img_masked = depth_img_orig * depth_img_mask
    depth_img_masked *= 255.0/depth_img_masked.max()
    depth_img_masked = np.uint8(depth_img_masked)
    edges = cv2.Canny(depth_img_masked,5,10)
    
    if(display_output):
        fig1 = plt.figure(figsize=(12,12))
        plt.title('Original Depth Image')
        plt.imshow(depth_img_orig, cmap='gray')
        plt.show()
        fig2 = plt.figure(figsize=(12,12))
        plt.title('Laplacian Filter applied to Depth Image')
        plt.imshow(edges_lap, cmap='gray')
        plt.show()
        fig3 = plt.figure(figsize=(12,12))
        plt.title('Laplacian Filter Converted to Binary')
        plt.imshow(edges_lap_binary, cmap='gray')
        plt.show()
        fig4 = plt.figure(figsize=(12,12))
        plt.title('Canny Filter Applied to Depth Image')
        plt.imshow(edges, cmap='gray')
        plt.show()

    if(save_output):
        imsave( edges_output_path%(i), edges_lap_binary )



# Test Edge Detection on Surface Normals

In [ ]:
%matplotlib inline 

import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from data_loader import Dataset
import OpenEXR, Imath
from scipy.misc import imsave
import imageio

class OPT():
    def __init__(self):
        self.dataroot = './data/'
        self.file_list = './data/datalist'
        self.batchSize = 32
        self.shuffle = True
        self.phase = 'train'
        self.num_epochs = 500
        self.imsize = 224
        self.num_classes = int(3)
        self.gpu = '0'
        self.logs_path = 'logs/exp9'
        self.use_pretrained = False

opt = OPT()

###################### DataLoader #############################
dataloader = Dataset(opt)
display_output = True
save_output = False

path_surface_normal_rgb = 'data/train/preprocessed-camera-normals/rgb-visualizations/%09d-cameraNormals.png'
edges_output_path = 'data/samples/edges-normals-imgs/%09d-normals-edges.jpg'

if not (os.path.isdir(os.path.dirname(path_surface_normal_rgb))):
    raise ValueError('The folder %s does not exist. Please check path.'%(os.path.dirname(path_surface_normal_rgb)))
if(save_output):
    if not (os.path.isdir(os.path.dirname(edges_output_path))):
        raise ValueError('The folder %s does not exist. Please check path.'%(os.path.dirname(edges_output_path)))

# TODO: LOAD IN SOURCE FILES AND CONVERT THEM INTO THE PRE-PROCESSED OUTPUT THIS CODE EXPECTS
for i in range (1,100):
    # Load Surface Normal RGB Img and Apply Blur
    print('Loading img %d'%(i))
    surface_normal_rgb = imageio.imread(path_surface_normal_rgb%(i))
#     surface_normal_gray = cv2.cvtColor(surface_normal_rgb, cv2.COLOR_BGR2GRAY)
#     surface_normal_gray = cv2.GaussianBlur(surface_normal_gray,(5,5),0)
    
    # Convert to HSV and then Grayscale
    surface_normal_hsv = cv2.cvtColor(surface_normal_rgb, cv2.COLOR_BGR2HSV)
    surface_normal_hsv = surface_normal_hsv[:,:,1]
    surface_normal_hsv = cv2.normalize(surface_normal_hsv, None, alpha=0, beta=255,
                                       norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    surface_normal_gray = cv2.GaussianBlur(surface_normal_hsv,(5,5),0)
    
    
    # Apply Laplacian filters for edge detection
    edges_lap = cv2.Laplacian(surface_normal_gray, cv2.CV_64F, ksize=3, borderType=1 )
    edges_lap = np.uint8(np.absolute(edges_lap)) 
    # Convert to binary 
    edges_lap_binary = edges_lap.copy()
    edges_lap_binary[edges_lap_binary>15] = 255
    edges_lap_binary[edges_lap_binary<=15] = 0
    
        
    # Apply Sobel Filter
    sobelx = cv2.Sobel(surface_normal_gray,cv2.CV_64F,1,0,ksize=1)
    sobely = cv2.Sobel(surface_normal_gray,cv2.CV_64F,0,1,ksize=1)
    sobelxy = sobelx + sobely
    sobelxy = np.uint8(np.absolute(sobelxy))
    # Convert to binary 
    edges_sobel_binary = sobelxy.copy()
    edges_sobel_binary[edges_sobel_binary>15] = 255
    edges_sobel_binary[edges_sobel_binary<=15] = 0
    
    
    if(display_output):
        fig0 = plt.figure(figsize=(12,12))
        plt.title('Surface Normals in RGB Representation')
        plt.imshow(surface_normal_rgb)
        plt.show()
        fig1 = plt.figure(figsize=(12,12))
        plt.title('Surface Normals in Grayscale (S channel out of HSV)')
        plt.imshow(surface_normal_gray, cmap='gray')
        plt.show()
#         fig2 = plt.figure(figsize=(12,12))
#         plt.title('Laplation Filter')
#         plt.imshow(edges_lap, cmap='gray')
#         plt.show()
#         fig3 = plt.figure(figsize=(12,12))
#         plt.title('Laplation filter converted to binary')
#         plt.imshow(edges_lap_binary, cmap='gray')
#         plt.show()
        fig4 = plt.figure(figsize=(12,12))
        plt.title('Sobel Filter Applied on X&Y axis to Grayscale img ')
        plt.imshow(sobelxy, cmap='gray')
        plt.show()
        fig5 = plt.figure(figsize=(12,12))
        plt.title('Sobel Filter Converted to Binary')
        plt.imshow(edges_sobel_binary, cmap='gray')
        plt.show()
    
    
    if(save_output):
        imsave( edges_output_path%(i), edges_lap_binary )



# Test the Pixel Values within Original depth2depth Weights examples,
# Generate and save own weights

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from PIL import Image 
import imageio
from matplotlib import interactive
interactive(True)
%matplotlib qt  
# %matplotlib inline 

# read in outlines/weights file
prefix = 'realsense_004'
path_outlines = 'data/depth2depth_sample_files/bound_realsense_test_bound/'+prefix+'_bound_est.png'
path_weights = 'data/depth2depth_sample_files/bound_realsense_weight/'+prefix+'_weight.png'
weight_save_path = 'data/depth2depth_sample_files/my_generated_weights/'+prefix+'_weight.png'

outlines = Image.open(path_outlines)
outlines = np.array(outlines)
weight_orig = Image.open(path_weights)
weight_orig = np.array(weight_orig)


# Create own weights
outlines = outlines.astype(np.float64)/255
weight = (1-outlines[:,:,1])
weight = np.power(weight,3)
weight = np.multiply(weight,1000)
weight = np.rint(weight)
weight = weight.astype(np.uint16)

print('The two weights are equal = ', np.array_equal(weight_orig, weight))
print('orig weights:\n', weight_orig)
print('my weights:\n', weight)

# Save my weights
array_buffer = weight.tobytes()
img = Image.new("I", weight.T.shape)
img.frombytes(array_buffer, 'raw', 'I;16')
img.save(weight_save_path)
weight_from_file = Image.open(weight_save_path)
weight_from_file = np.array(weight_from_file)
print('My gen weights and weights from file are equal: ', np.array_equal(weight, weight_from_file))

# Create interactive plot to analyze pixel values
fig = plt.figure(figsize=(8, 6))
ax0 = plt.subplot(141)
ax1 = plt.subplot(142)
ax2 = plt.subplot(143)
ax3 = plt.subplot(144)

ax0.imshow(outlines)
ax0.set_title('Predicted Outlines')
ax1.imshow(weight_orig)
ax1.set_title('Orig Weights from \n Outlines')
ax2.imshow(weight)
ax2.set_title('My Weights from \n Outlines')
ax3.imshow(weight_from_file)
ax3.set_title('My Saved Weights')
    
plt.show()


    